In [2]:
SPLITTED_DATASET = "DATASET"
train_dir = "DATASET/train"
test_dir  = "DATASET/test"

train_dir,test_dir

('DATASET/train', 'DATASET/test')

#### Visualize train_dir, test_dir

In [ ]:
import random
from pathlib import Path
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Set seed
# random.seed(30) # <- try changing this and see what happens
SPLITTED_DATASET = "DATASET"

# Convert SAVE_DATASET_PATH to a Path object
save_dataset_path = Path(SPLITTED_DATASET)

# 1. Get all image paths (* means "any combination")
image_extensions = ["*.jpg", "*.jpeg", "*.JPG", "*.png", "*.PNG"]
image_path_list = []

for ext in image_extensions:
    image_path_list.extend(save_dataset_path.rglob(ext))

# Check if the list is empty
if not image_path_list:
    print("No images found in the specified path.")
else:
    # 2. Get random image path
    random_image_path = random.choice(image_path_list)

    # 3. Get image class from path name (the image class is the name of the directory where the image is stored)
    image_class = random_image_path.parent.stem

    # 4. Open image
    img = Image.open(random_image_path)

    # 5. Print metadata
    print(f"Random image path: {random_image_path}")
    print(f"Image class: {image_class}")
    print(f"Image height: {img.height}") 
    print(f"Image width: {img.width}")
    # img.show()  # Display the image

    # Turn the image into an array
    img_as_array = np.asarray(img)

    # Plot the image with matplotlib
    plt.figure(figsize=(10, 7))
    plt.imshow(img_as_array)
    plt.title(f"Image class: {image_class} | Image shape: {img_as_array.shape} -> [height, width, color_channels]")
    plt.axis(False);

#### Yolo

In [ ]:
from ultralytics import YOLO
import torch

model = 'yolov8n-cls.pt'

def load_model_on_cuda(model):
    # Load the model
    model = YOLO(model)
    
    # Check if CUDA is available
    if torch.cuda.is_available():
        device = torch.device('cuda')
        model.to(device)
        print(f"Model moved to {device}")
    else:
        device = torch.device('cpu')
        print(f"CUDA not available, using {device}")

    return model

model = load_model_on_cuda(model)